# Hop-pub functions

Sebastian Torres-Lara<br>
Melih Kara<br>
30-08-2021

Todo: 
- Check how the heartbeat runs in the background. 
- Check if `SNEWSObservation` and `SNEWSHeartbeat` are good enough or if they can be improved <br>
- ~For me (Melih) publising often fails (simple hop publish) for some reason. Need to investigate.~

~Should we implement the following format?~ I do not like it. 
```python
return SNEWSObservation(message_id=str(uuid.uuid4()),
                        detector_id=self.detector.id,
                        sent_time=self.time_str(), 
                        neutrino_time=self.time_str(), 
                        machine_time=self.time_str(), 
                        location=self.detector.location,
                        p_value=0.5,
                        status="none",
                        content="none").asdict()
```
SNEWSObservation requires all fields to be filled for the different piers the idea is the opposite. <br>
we should be able to not-return some of the fields. <br>

**-** Maybe fill the empty sections with `'none'` ?

## Application

In [10]:
from hop_pub import Publish_Observation

First create an object with the information. <br>
Below, the `Publish_Observation` can have 
- `message` argument as a dictionary, `SNEWSObservation` or `SNEWSHeartbeat` objects, a file path containing a json or a dictionary. <br>
- `detector` argument can also be specified in which case the object knows its ID and location. If not given, it takes a detector called 'TEST'. <br>
- `env_path` as a string containing the broker, time formatting and other information. If not specified, uses the deafult environment 'test_env.env'<vr>
- If `welcome` argument is set, it displays the versions and the connected topics.

In [11]:
publisher = Publish_Observation(detector='XENONnT', welcome=True)

##################################################
#       Publish SNEWS OBSERVATION Messages       #
#____________________XENONnT_____________________#
#_________________19-loc XENONnT_________________#
##################################################
Your Python version:
 3.7.11 (default, Jul 27 2021, 14:32:16) 
[GCC 7.5.0]
Current hop-client version:0.4.0
             snews version:0.0.1

Publishing to kafka.scimma.org
Observation Topic:
==> kafka://kafka.scimma.org/snews.experiments-test
Heartbeat Topic:
==> kafka://kafka.scimma.org/snews.experiments-test




In [3]:
publisher.tier_keys_['Significance_Tier']

['detector_id',
 'machine_time',
 'neutrino_time',
 'sent_time',
 'status',
 'p_value']

### See & Modify the message before publishing

If no message fed in the beginning, it produces a default one like the following. <br>
One can add, or modify the fields.

In [4]:
publisher.display_message()

Following OBS message to be published:
Current time:21/08/30 19:33:28

message_id          :19_O_21/08/30_19:33:26
detector_id         :19
sent_time           :21/08/30 19:33:26
neutrino_time       :21/08/30 19:33:26
machine_time        :21/08/30 19:33:26
location            :loc XENONnT
p_value             :0
status              :none
content             :none

> modify self.message_dict or 
> use .publish_to_tiers() method to publish (see .publish_to)


In [5]:
# modify the content and add a non-existing field
publisher.message_dict['content'] = 'This is a modified content'
publisher.message_dict['New Field'] = 'This is a new field'
publisher.display_message()

Following OBS message to be published:
Current time:21/08/30 19:33:29

message_id          :19_O_21/08/30_19:33:26
detector_id         :19
sent_time           :21/08/30 19:33:26
neutrino_time       :21/08/30 19:33:26
machine_time        :21/08/30 19:33:26
location            :loc XENONnT
p_value             :0
status              :none
content             :This is a modified content
New Field           :This is a new field

> modify self.message_dict or 
> use .publish_to_tiers() method to publish (see .publish_to)


### Submission to Different Tiers

Here, it is not clear to me conceptually. There is only one topic that the experiments can publish their messages. <br>
The different Tiers should be based on the content of the messages. Currently, the script does this by taking only the fields relevant for that tier. See ``

In [6]:
# See which tiers are connected
print(publisher.publish_to)
publisher.publish_to['Timing_Tier'] = False
print(publisher.publish_to)

{'Significance_Tier': True, 'Coincidence_Tier': True, 'Timing_Tier': True}
{'Significance_Tier': True, 'Coincidence_Tier': True, 'Timing_Tier': False}


In [12]:
publisher.publish_to_tiers()


Publishing OBS message to Significance_Tier:
detector_id         :19
machine_time        :21/08/30 19:40:05
neutrino_time       :21/08/30 19:40:05
sent_time           :21/08/30 19:40:12
status              :none
p_value             :0

Publishing OBS message to Coincidence_Tier:
detector_id         :19
machine_time        :21/08/30 19:40:05
neutrino_time       :21/08/30 19:40:05
sent_time           :21/08/30 19:40:16
status              :none

Publishing OBS message to Timing_Tier:
detector_id         :19
machine_time        :21/08/30 19:40:05
neutrino_time       :21/08/30 19:40:05
sent_time           :21/08/30 19:40:19
status              :none
content             :none


---
**Publish the message**

In [9]:
publisher.publish()


Published the Heartbeat message to kafka://kafka.scimma.org/snews.experiments-test:
detector_id         :19
message_id          :19_H_21/08/30_19:39:32
status              :ON
sent_time           :21/08/30 19:39:32


---

# Heartbeat Messages

In [13]:
from hop_pub import Publish_Heartbeat
publisher = Publish_Heartbeat(detector='XENONnT', rate=10)

Press Ctrl+C to exit

Published the Heartbeat message to kafka://kafka.scimma.org/snews.experiments-test:
detector_id         :19
message_id          :19_H_21/08/30_19:40:53
status              :OFF
sent_time           :21/08/30 19:40:53

Published the Heartbeat message to kafka://kafka.scimma.org/snews.experiments-test:
detector_id         :19
message_id          :19_H_21/08/30_19:41:03
status              :OFF
sent_time           :21/08/30 19:41:03

Published the Heartbeat message to kafka://kafka.scimma.org/snews.experiments-test:
detector_id         :19
message_id          :19_H_21/08/30_19:41:13
status              :ON
sent_time           :21/08/30 19:41:13


---

# Alert Messages

In [14]:
from hop_pub import Publish_Alert
alert_publisher = Publish_Alert()
alert_publisher.publish()


Published ALERT message to kafka://kafka.scimma.org/snews.alert-test !!!
